Customers who use Snowflake with AzureML and Sagemaker will have models registered in either cloud ML tool and prefer to deploy those models to Snowflake. Often it's the case that all (or most) of the inference data is in Snowflake and customers prefer the ease of use, performance and security of having the model deployed in Snowflake.

For models registered in the AWS Sagemaker model registry you can use the below code to access the model from the Sagemaker registry and push it to the Snowflake registry.

Important Notes

For both AzureML and Sagemaker deployment patterns users will need to make sure that they have properly installed all of the necessary libraries in their Python environments. For the required Snowflake packages this can usually be done with the instructions here: https://docs.snowflake.com/en/developer-guide/snowpark-ml/index#installing-snowpark-ml

Additionally, when using log_model() to push the model into the Snowflake registry users may need to specific dependencies and versions. This can be done with an additional argument in the function. Please see the documentation here: https://docs.snowflake.com/en/developer-guide/snowpark-ml/reference/latest/api/registry/snowflake.ml.registry.Registry#snowflake.ml.registry.Registry

Lastly, users will want to make sure that their model is supported by libraries that are included in the Snowflake conda channel. https://repo.anaconda.com/pkgs/snowflake/
If the model is not supported then users will need to consider installing the libraries as 3rd party packages or using Snowpark Container Services to deploy their model as an application. https://docs.snowflake.com/en/developer-guide/snowpark-container-services/overview

With Sagemaker there is no way easy programmatic way to access the dependencies associated with a registered model. Customers are advised to be aware of the libraries used to build/train models so they can better understand what is required to deploy models to Snowflake. If customers are unaware of those dependencies they can reference the Sagemaker prebuilt docker containers here to review the source code and requirements. 
https://docs.aws.amazon.com/sagemaker/latest/dg/docker-containers-prebuilt.html

In [ ]:
# import packages
import boto3
from sagemaker import get_execution_role
import sagemaker
from joblib import load

# create clients for S3 and sagemaker
s3 = boto3.client("s3")
sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()

In [ ]:
# using model arn from model registry describe model details
sm_client = boto3.Session().client('sagemaker')
sm_client.describe_model_package(ModelPackageName='<model arn>')

In [ ]:
# download model from S3 using information from above
s3_client = boto3.client('s3')
s3_client.download_file('<bucket from ModelDataUrl>', 'model file path from ModelDataURL', '<new model name in local sagemaker env>.tar.gz')

In [ ]:
# unzip and load model
!tar -xf '<new model name in local sagemaker env>.tar.gz' -C .

In [ ]:
# load model
sm_model = load("model.joblib")
sm_model

In [ ]:
# connect to Snowflake with Snowpark
import pandas as pd
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
from snowflake.ml.registry import registry

connection_parameters = {
    "account": "<locator>", # e.g. xy12345.us-east-2.aws 
    "user": "<username>", 
    "password": "<password>",
    "role": "<role>",
    "warehouse": "<vistual warehouse>",
    "database": "<database>",
    "schema": "<schema>"
    }
session = Session.builder.configs(connection_parameters).create()

In [ ]:
# connect to Snowpark registry and log model
reg = registry.Registry(session=session)
reg.log_model(sm_model, model_name='<name of model in Snowflake>', version_name='v1', sample_input_data=<sample dataframe>)

In [ ]:
# verify model deployment and view functions associated with the model
mv = reg.get_model('<name of model in Snowflake>').version('v1')
mv.show_functions()